# 实现RNN的基本单元——RNN Cell

在Tensorflow中定义一个基本的RNN单元

In [ ]:
import tensorflow as tf
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=128)
print(rnn_cell.state_size) # 打出state_size看一下，应为128

在Tensorflow中定义一个基本的LSTM单元

In [ ]:
import tensorflow as tf
lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=128)
print(lstm_cell.state_size) # 打出state_size看一下，应为128

一个简单的例子

In [ ]:
# 新版本没有call方法，可以调用这个查看output,h1 = lstm_cell.__call__(input,h0) 
import tensorflow as tf
import numpy as np

lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=128)
inputs = tf.placeholder(np.float32, shape=(32,100))
h0 = lstm_cell.zero_state(32,np.float32)
output, h1 = lstm_cell.__call__(inputs, h0)

print(h1.h)
print(h1.c)

In [ ]:
import tensorflow as tf
import numpy as np

lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=128)
inputs = tf.placeholder(np.float32, shape=(32,100))
h0 = lstm_cell.zero_state(32,np.float32)
output, h1 = lstm_cell(inputs, h0)

print(h1.h)
print(h1.c)

# 对RNN进行堆叠——MultiRNNCell

多层RNN

In [2]:
import tensorflow as tf
import numpy as np

# 每调用一次这个函数返回一个BasicRNNCell
def get_a_cell():
    return tf.nn.rnn_cell.BasicRNNCell(num_units=128)

# 用 tf.nn.rnn_cell.MultiRNNCell 创建3层RNN
cell = tf.nn.rnn_cell.MultiRNNCell( [get_a_cell() for _ in range(3)] )

# 得到的cell实际也是 RNNCell的子类
# (128, 128, 128) 不是128x128x128的意思，而是表示共有3个隐层状态，每个隐层状态的大小为128
print(cell.state_size)

# 使用对应的 call 函数
inputs = tf.placeholder(np.float32, shape=(32,100))
h0 = cell.zero_state(32,np.float32)
output, h1 = cell.__call__(inputs, h0)
print(h1)

(128, 128, 128)
(<tf.Tensor 'multi_rnn_cell/cell_0/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'multi_rnn_cell/cell_1/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'multi_rnn_cell/cell_2/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>)


# 使用tf.nn.dynamic_rnn展开时间维度

对于单个的RNNCell，使用它的call函数进行运算时，只是在序列时间上前进了一步。如使用x1, h0 得到h1

如果序列长度为n，要调用n次call函数，比较麻烦。因此，用dynamic实现
通过{h1, x1, x2, ..., xn} 直接得到 { h1, h2, ..., hn }

In [ ]:
# 给出语句结构，不是直接运行
# inputs shape = ( batch_size, time_steps, input_size )
# cell: RNNCell
# initial_state: shape = ( batch_size, cell.state_size )  初始状态，一般可以取零矩阵
outputs, state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state)

# 学习使用Tensorflow实现CharRNN实现（未完成）

运行环境为Python2.7以上，Tensorflow1.2以上，

《model.py》
self.inputs 是外部传入的一个batch内的输入数据，它的形状为( self.num_seqs, self.num_steps )，其
            中self.num_seqs表示一个batch内句子的个数， self.num_steps 表示每个句子的长度
self.targets 是 self.inputs 对应的训练目标，它的形状和 self.inputs相同，内容是 self.inputs 每个字母对应的下             一个字母。
self.keep_prob 是因为后面的模型有Dropout层，控制Dropout层所需要的概率。训练时，使用 self.keep_prob=0.5，
            测试时，使用 self.keep_prob=1.0

In [ ]:
def build_inputs(self):
    with tf.name_scope('inputs'):
        self.inputs = tf.placeholder(tf.int32, shape=(
            self.num_seqs, self.num_steps), name='inputs')
        self.targets = tf.placeholder(tf.int32, shape=(
            self.num_seqs, self.num_steps), name='targets')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')

        # 对于中文，需要使用embedding层
        # 英文字母没有必要用embedding层
        if self.use_embedding is False:
            self.lstm_inputs = tf.one_hot(self.inputs, self.num_classes)
        else:
            with tf.device("/cpu:0"):
                embedding = tf.get_variable('embedding', [self.num_classes, self.embedding_size])
                self.lstm_inputs = tf.nn.embedding_lookup(embedding, self.inputs)


下面的函数定义了多层的 N V N LSTM模型

In [ ]:
def build_lstm(self):
    # 创建单个cell并堆叠多层
    def get_a_cell(lstm_size, keep_prob):
        lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
        drop = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop

    with tf.name_scope('lstm'):
        cell = tf.nn.rnn_cell.MultiRNNCell(
            [get_a_cell(self.lstm_size, self.keep_prob) for _ in range(self.num_layers)]
        )
        self.initial_state = cell.zero_state(self.num_seqs, tf.float32)

        # 通过dynamic_rnn对cell展开时间维度
        self.lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cell, self.lstm_inputs, initial_state=self.initial_state)

        # 通过lstm_outputs得到概率
        seq_output = tf.concat(self.lstm_outputs, 1)
        x = tf.reshape(seq_output, [-1, self.lstm_size])

        with tf.variable_scope('softmax'):
            softmax_w = tf.Variable(tf.truncated_normal([self.lstm_size, self.num_classes], stddev=0.1))
            softmax_b = tf.Variable(tf.zeros(self.num_classes))

        self.logits = tf.matmul(x, softmax_w) + softmax_b
        self.proba_prediction = tf.nn.softmax(self.logits, name='predictions')


定义损失

In [ ]:
def build_loss(self):
    with tf.name_scope('loss'):
        y_one_hot = tf.one_hot(self.targets, self.num_classes)
        y_reshaped = tf.reshape(y_one_hot, self.logits.get_shape())
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=y_reshaped)
        self.loss = tf.reduce_mean(loss)

以上是模型从输入到损失的全过程

《 训练模型与生成文字过程 》

首先来看简单的生成英文的例子。使用的训练文件 shakespeare.txt 放置在data/文件夹下

In [ ]:
训练命令为
python train.py \
    --input_file data/shakespeare.txt \
    --name shakespeare \
    --num_steps 50 \
    --num_seqs 32 \
    --learning_rate 0.01 \
    --max_steps 20000 \